In [2]:
import rqalpha as rqa
import rqalpha_plus
import rqdatac as rqd
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm
import utils

In [3]:
pd.set_option('display.width',200)
pd.set_option('display.max_columns', 200)

In [4]:
rqd.init()

F:\anaconda\envs\ricequant\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


# 1.获得因子数据

In [5]:
start_date = '20140301'
end_date = '20200929'

In [29]:
category = ['业绩增长','业绩增长','业绩增长','投资回报率','投资回报率','规模增长','效率提升','效率提升','效率提升']
factors_name_dict = {
    '营收增长':'inc_revenue_ttm',
    '毛利润增长':'gross_profit_growth_ratio_ttm',
    '净利润增长':'net_profit_growth_ratio_ttm',
    '投入资本回报率':'return_on_invested_capital_ttm',
    '股权投资回报率':'return_on_equity_ttm',
    '总资产增长':'total_asset_growth_ratio_ttm',
    '毛利率':'gross_profit_margin_ttm',
    '经营利润率':'net_profit_to_revenue_ttm',
    '总资产周转率':'total_asset_turnover_ttm'
}

In [30]:
columns = pd.MultiIndex.from_arrays([category,factors_name_dict.keys()])

In [31]:
dates = rqd.get_trading_dates(start_date,end_date)

# 每月底交易日
dates_monthly = []
lastdate = None
for date in dates:
    if lastdate is None:
        lastdate = date
        continue
    if lastdate.month != date.month:
        dates_monthly.append(lastdate)
    lastdate = date

In [43]:
df_list = []
for date in tqdm(dates_monthly):
    universe =  rqd.all_instruments(type='CS',date=date)['order_book_id'].tolist()
    df = rqd.get_factor(universe,factors_name_dict.values(),start_date=date,end_date=date,expect_df=True)
    df_list.append(df.dropna())
data = pd.concat(df_list)
data.columns = columns

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [10:18<00:00,  7.92s/it]


In [45]:
data = data.swaplevel(0,1).sort_index()

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 223285 entries, (Timestamp('2014-03-31 00:00:00'), '000002.XSHE') to (Timestamp('2020-08-31 00:00:00'), '603999.XSHG')
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   (业绩增长, 营收增长)      223285 non-null  float64
 1   (业绩增长, 毛利润增长)     223285 non-null  float64
 2   (业绩增长, 净利润增长)     223285 non-null  float64
 3   (投资回报率, 投入资本回报率)  223285 non-null  float64
 4   (投资回报率, 股权投资回报率)  223285 non-null  float64
 5   (规模增长, 总资产增长)     223285 non-null  float64
 6   (效率提升, 毛利率)       223285 non-null  float64
 7   (效率提升, 经营利润率)     223285 non-null  float64
 8   (效率提升, 总资产周转率)    223285 non-null  float64
dtypes: float64(9)
memory usage: 16.2+ MB


In [47]:
data

业绩增长                         投资回报率                 规模增长      效率提升                     
                              营收增长     毛利润增长     净利润增长   投入资本回报率    股权投资回报率     总资产增长       毛利率      经营利润率    总资产周转率
date       order_book_id                                                                                            
2014-03-31 000002.XSHE    0.313263  0.157450  0.168233  0.105831   0.223276  0.309391  0.314736   0.135118  0.302531
           000004.XSHE    0.693459  0.320792  0.435734  0.073777   0.042768  0.011636  0.452795   0.112038  0.532575
           000005.XSHE    0.279304 -0.655366 -1.410064  0.031939   0.009622 -0.034773  0.421057   0.075148  0.070897
           000006.XSHE    0.498858  0.094756  0.108716  0.117094   0.188787  0.174016  0.332134   0.151384  0.448457
           000007.XSHE   -0.232473 -1.796984  0.795274  0.081838   0.130923  0.348132  0.569055   0.151570  0.307858
...                            ...       ...       ...       ...        ...       ...       ...        ...       ...
2020-08-31 603993.XSHG    3.818711 -1.894836 -0.141264  0.032201   0.050825  0.183162  0.014899   0.018916  0.869850
           603996.XSHG   -0.967082  6.227532  8.869926 -1.246634 -27.718983 -0.436152 -0.169818 -18.623825  0.045809
           603997.XSHG    0.016430 -2.345920 -2.403677  0.000954  -0.061536  1.199799  0.126242  -0.018506  1.158268
           603998.XSHG    0.052172 -0.440156 -0.340400  0.041439   0.056087  0.169469  0.675234   0.049889  0.568743
           603999.XSHG    0.520029 -0.596263  0.343742  0.030149   0.039515  0.035870  0.182527   0.061039  0.512110

[223285 rows x 9 columns]

In [48]:
data.to_csv('成长因子数据.csv')